# It definitely came as a surprise to me that there was this huge shake up in the end. 
I personally thought that the shake up should be relatively small because my stacking 
model performance on public LB and local cv was always aligned although for single model 
they could vary quite a bit. We did trust mostly on local CV and we tried very hard on nonlinear 
stacking (without success) so we relied on weighted average cv score and only submitted when there was an improvement on local cv.

# I cannot wait to see what @Michael Jahrer did but I would like to share some of our approaches (actually, just the NN part, and leave the rest to my teammates) before Michael publishes his and then nobody cares what we did :(

# So my best NN had a 5 fold local CV 0.294, and public LB 0.284 and private 0.290. And here is roughly how you can achieve that:

# 1) important features' interactions (e.g. ps_car_13, ps_ind_03, ps_reg_03, ...)

# 2) count of categorical features

# 3) xgboost predictions: divide feature sets into three groups (car, ind, reg) and then use two group as features and the other group as target, train a xgboost model on it, and use prediction as features

# 4) feature aggregation: pick two features (e.g. ps_car_13, ps_ind_03), and then use one as group variable, the other as value variable, do mean, std, max, min, median. Still top important features are picked only

# 5) do embedding layers on all categorical features (I used embedding dimension 4 with dropout 0.25)

# 6) model is 2 layer with relu activation and big dropout (512 with dropout 0.8, and 64 with 0.8)

# 7) a few categorical count features @qianqian created

# That is about it! Hope it helps :)

# Update: my best lightgbm code is here

# https://www.kaggle.com/xiaozhouwang/2nd-place-lightgbm-solution

# It scored 0.29124 on private LB.

# best nn code is here https://www.kaggle.com/xiaozhouwang/2nd-place-solution-nn-model

# It scored 0.29008 on private LB.

# And their model results can be found in the comments (Seems I cannot upload files here?)

In [1]:
# part of 2nd place solution: lightgbm model with private score 0.29124 and public lb score 0.28555

import lightgbm as lgbm
from scipy import sparse as ssp
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def Gini(y_true, y_pred):
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]

    # sort rows on prediction column
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:, 0].argsort()][::-1, 0]
    pred_order = arr[arr[:, 1].argsort()][::-1, 0]

    # get Lorenz curves
    L_true = np.cumsum(true_order) * 1. / np.sum(true_order)
    L_pred = np.cumsum(pred_order) * 1. / np.sum(pred_order)
    L_ones = np.linspace(1 / n_samples, 1, n_samples)

    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)

    # normalize to true Gini coefficient
    return G_pred * 1. / G_true

cv_only = True
save_cv = True
full_train = False

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', Gini(labels, preds), True

path = "../data/"

train = pd.read_csv(path+'train.csv')
train_label = train['target']
train_id = train['id']
test = pd.read_csv(path+'test.csv')
test_id = test['id']

NFOLDS = 5
kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=218)

y = train['target'].values
drop_feature = [
    'id',
    'target'
]

X = train.drop(drop_feature,axis=1)
feature_names = X.columns.tolist()
cat_features = [c for c in feature_names if ('cat' in c and 'count' not in c)]
num_features = [c for c in feature_names if ('cat' not in c and 'calc' not in c)]

train['missing'] = (train==-1).sum(axis=1).astype(float)
test['missing'] = (test==-1).sum(axis=1).astype(float)
num_features.append('missing')

for c in cat_features:
    le = LabelEncoder()
    le.fit(train[c])
    train[c] = le.transform(train[c])
    test[c] = le.transform(test[c])

enc = OneHotEncoder()
enc.fit(train[cat_features])
X_cat = enc.transform(train[cat_features])
X_t_cat = enc.transform(test[cat_features])

ind_features = [c for c in feature_names if 'ind' in c]
count=0
for c in ind_features:
    if count==0:
        train['new_ind'] = train[c].astype(str)+'_'
        test['new_ind'] = test[c].astype(str)+'_'
        count+=1
    else:
        train['new_ind'] += train[c].astype(str)+'_'
        test['new_ind'] += test[c].astype(str)+'_'

cat_count_features = []
for c in cat_features+['new_ind']:
    d = pd.concat([train[c],test[c]]).value_counts().to_dict()
    train['%s_count'%c] = train[c].apply(lambda x:d.get(x,0))
    test['%s_count'%c] = test[c].apply(lambda x:d.get(x,0))
    cat_count_features.append('%s_count'%c)

train_list = [train[num_features+cat_count_features].values,X_cat,]
test_list = [test[num_features+cat_count_features].values,X_t_cat,]

X = ssp.hstack(train_list).tocsr()
X_test = ssp.hstack(test_list).tocsr()

learning_rate = 0.1
num_leaves = 15
min_data_in_leaf = 2000
feature_fraction = 0.6
num_boost_round = 10000
params = {"objective": "binary",
          "boosting_type": "gbdt",
          "learning_rate": learning_rate,
          "num_leaves": num_leaves,
          "max_bin": 256,
          "feature_fraction": feature_fraction,
          "verbosity": 0,
          "drop_rate": 0.1,
          "is_unbalance": False,
          "max_drop": 50,
          "min_child_samples": 10,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "subsample": 0.9
          }

x_score = []
final_cv_train = np.zeros(len(train_label))
final_cv_pred = np.zeros(len(test_id))
for s in range(16):
    cv_train = np.zeros(len(train_label))
    cv_pred = np.zeros(len(test_id))

    params['seed'] = s

    if cv_only:
        kf = kfold.split(X, train_label)

        best_trees = []
        fold_scores = []

        for i, (train_fold, validate) in enumerate(kf):
            X_train, X_validate, label_train, label_validate = \
                X[train_fold, :], X[validate, :], train_label[train_fold], train_label[validate]
            dtrain = lgbm.Dataset(X_train, label_train)
            dvalid = lgbm.Dataset(X_validate, label_validate, reference=dtrain)
            bst = lgbm.train(params, dtrain, num_boost_round, valid_sets=dvalid, feval=evalerror, verbose_eval=100,
                            early_stopping_rounds=100)
            best_trees.append(bst.best_iteration)
            cv_pred += bst.predict(X_test, num_iteration=bst.best_iteration)
            cv_train[validate] += bst.predict(X_validate)

            score = Gini(label_validate, cv_train[validate])
            print(score)
            fold_scores.append(score)

        cv_pred /= NFOLDS
        final_cv_train += cv_train
        final_cv_pred += cv_pred

        print("cv score:")
        print(Gini(train_label, cv_train))
        print("current score:", Gini(train_label, final_cv_train / (s + 1.)), s+1)
        print(fold_scores)
        print(best_trees, np.mean(best_trees))

        x_score.append(Gini(train_label, cv_train))

print(x_score)
pd.DataFrame({'id': test_id, 'target': final_cv_pred / 16.}).to_csv('../cache/lgbm3_pred_avg.csv', index=False)
pd.DataFrame({'id': train_id, 'target': final_cv_train / 16.}).to_csv('../cache/lgbm3_cv_avg.csv', index=False)

Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's gini: 0.279467
[200]	valid_0's gini: 0.285768
[300]	valid_0's gini: 0.287527
Early stopping, best iteration is:
[263]	valid_0's gini: 0.287886
0.287885620147
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's gini: 0.285865
[200]	valid_0's gini: 0.288072
[300]	valid_0's gini: 0.287674
Early stopping, best iteration is:
[257]	valid_0's gini: 0.288528
0.28852776919
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's gini: 0.276976
[200]	valid_0's gini: 0.278518
[300]	valid_0's gini: 0.276621
Early stopping, best iteration is:
[211]	valid_0's gini: 0.278608
0.278608196168
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's gini: 0.280307
[200]	valid_0's gini: 0.285361
[300]	valid_0's gini: 0.285197
Early stopping, best iteration is:
[246]	valid_0's gini: 0.286498
0.286498445195
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's gini: 0.28769


In [2]:
sum([0.28659822049621808, 0.28625403977021635, 0.28506522804500412, 0.28591388431383108, 0.28578218931010158, 0.28650884427189527, 0.28657589721670779, 0.28623883067773614, 0.28594786256612847, 0.28644462304904134, 0.28576628627213485, 0.28661170541466702, 0.28658951282259731, 0.28718514775798354, 0.28640557081265966, 0.28554157235270489])/16


0.2862143384468517